We begin by making appropriate imports as well as loading the data needed for NLTK and Spacy.

In [54]:
import math
import os
import random

from pprint import pprint
from typing import List, Dict

import nltk
import numpy as np
import spacy
# Download the required dataset from NLTK
nltk.download("stopwords", quiet=True)

from nltk.corpus import stopwords
from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

# If this fails, please run `python -m spacy download en_core_web_sm`
nlp = spacy.load("en_core_web_sm")

Two functions can then be defined to load the data from the text files to memory

In [55]:
def load_corpus(folder: str) -> List[str]:
    """Load strings from folder of text

    Args:
        folder (str): The path to the folder to load

    Returns:
        List[str]: List of strings retrieved from text files in the folder
    """
    corpus = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            try:
                with open(os.path.join(root, name), "r") as fp:
                    corpus.append(fp.read())
            except UnicodeDecodeError as e:
                ... # Let the error pass silently
                # print(e.__str__(), "for", os.path.join(root, name))
    return corpus

def load_corpuses(folder: str) -> Dict[str, List[str]]:
    """Load corpuses from sub-folders of specified folder

    Args:
        folder (str): The parent folder

    Returns:
        Dict[str, List[str]]: Dictionary of corpuses
    """
    sub_folders = []
    for root, dirs, files in os.walk(folder):
        if dirs:
            for dir_ in dirs:
                sub_folders.append(dir_)

    corpuses = {}
    for sub_folder in sub_folders:
        corpuses[sub_folder] = load_corpus(os.path.join(folder, sub_folder))
    return corpuses

We then build our dataset using these corpuses. The $x$ vector is made from three features:
- Word frequencies
- Frequency of named entity types
- Weighted word frequencies

Once constructed, the data set is shuffled.

In [56]:
x = []
y =[]

corpuses = load_corpuses("bbc")

for corpus in corpuses:
    for story in corpuses[corpus]:
        x.append(story)
        y.append(corpus)
c = list(zip(x, y))

random.shuffle(c)

x, y = zip(*c)

Test and training data are then sampled using a 20:80 split respectively. The $Y$ values are then encoded so that they can be used as labels within the SVM.

In [57]:
size_dataset_full=len(x)
size_test=int(round(size_dataset_full*0.2,0))

list_test_indices=random.sample(range(size_dataset_full), size_test)

test_x = []
test_y = []
train_x = []
train_y = []

for i,example in enumerate(x):
    if i in list_test_indices:
        test_x.append(example)
        test_y.append(y[i])
    else:
        train_x.append(example)
        train_y.append(y[i])

le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

In [58]:
def feature_extraction(stories: List[str]) -> List[List[int]]:
    """Extracts features from a list of strings

    Args:
        stories (List[str]): Strings to extract features from

    Returns:
        List[List[int]]: List of vectors which can be used in a model
    """
    entity_types = CountVectorizer(stop_words=stopwords.words('english'))
    entity_types.fit(['CARDINAL', 'PERSON', 'GPE', 'MONEY', 'ORG', 'ORDINAL', 'WORK_OF_ART', 'NORP', 'PERCENT', 'DATE', 'LANGUAGE', 'FAC', 'LOC', 'TIME', 'PRODUCT', 'EVENT', 'QUANTITY', 'LAW'])
    processed_stories = []
    for story in tqdm(stories):
        analysed = nlp(story)
        processed_stories.append(
            list(vectorizer.transform([story]).toarray()[0]) + 
            list(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0]) +
            list(tfid.transform(vectorizer.transform([story])).toarray()[0])
        )
    return processed_stories

We then define two of the feature extraction methods. `CountVectorizer` builds a vocabulary from the previously loaded training data. `TfidfTransformer` is then built using the matrix provided by `CountVectorizer`.

The combination of the vectors resulted in very large $x$ vectors to train on. The best 500 features are selected using the $\chi^2$ method

In [59]:
vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
tfid = TfidfTransformer()

vectorizer.fit(train_x)
tfid.fit(vectorizer.transform(train_x))

train_x = feature_extraction(train_x)
print(type(train_x))

get_best=SelectKBest(chi2, k=500).fit(train_x, train_y)
train_x_chi = get_best.transform(train_x)

  0%|          | 0/1779 [00:00<?, ?it/s]

<class 'list'>


The SVM object is constructed. The pipeline includes passing the data through the `StandardScaler` function which "Standardize \[sic\] features by removing the mean and scaling to unit variance".

In [60]:
svm_clf=make_pipeline(StandardScaler(), svm.SVC(cache_size=10000, decision_function_shape='ovo'))

We can then pass the training data to the SVM to train the model.

In [61]:
svm_clf.fit(train_x_chi, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(cache_size=10000, decision_function_shape='ovo'))])

Using this model, we can run the training data through it in order to evaluate the SVM.

In [62]:
Y_text_predictions = svm_clf.predict(get_best.transform(feature_extraction(test_x)))

  0%|          | 0/445 [00:00<?, ?it/s]

The `classification_report` function allows us to easily generate a report on the success of the SVM by providing known good $Y$ values as well as $Y$ values attained through the SVM.

In [63]:
print(classification_report(test_y, Y_text_predictions, target_names=le.inverse_transform(svm_clf.classes_)))

               precision    recall  f1-score   support

     business       0.88      0.92      0.90       100
entertainment       0.97      0.88      0.92        76
     politics       0.99      0.90      0.94        93
        sport       0.90      0.98      0.94       111
         tech       0.89      0.89      0.89        65

     accuracy                           0.92       445
    macro avg       0.93      0.92      0.92       445
 weighted avg       0.92      0.92      0.92       445



In order to use the SVM, a large amount of preprocessing needs to be done on a string. This has been encapsulated in the following function:

In [64]:
def predict(story: str) -> str:
    """Gives a genre prediction for a news story

    Args:
        story (str): The plaintext of the story

    Returns:
        str: The genre of the story
    """
    return le.inverse_transform(
        svm_clf.predict(
            get_best.transform(
                feature_extraction([story])
            )
        )
    )[0]

We can then try this function with a news story.

In [65]:
predict("""
This guide is a concise summary of the main policies being put forward by each party.

The policy areas featured in the guide were selected using polling data on what the public consider to be the most important issues facing the country.

While some issues such as health and education are the responsibility of the Scottish Parliament, others such as foreign policy and Brexit are decided at the UK parliament at Westminster. You can read more about how devolution works here.

More information on how the issues and parties were selected is in our methodology.

A full list of parties standing at the election will be published after nominations have closed.
""")

  0%|          | 0/1 [00:00<?, ?it/s]

'sport'